# Data import from the "Congreso de los Diputados" website

This process is focused on extracting the political debates that happened in the congress and tag every intervention with the name of the politician. The main objective is to create a corpora for political profiles that can be used to train an ML Transformer.

In [1]:
# Imports
from html.parser import HTMLParser
import re
import requests
import urllib3
import html
import pandas as pd

# constants
page = 'https://www.congreso.es/busqueda-de-publicaciones?p_p_id=publicaciones&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&_publicaciones_mode=mostrarTextoIntegro&_publicaciones_legislatura=XII&_publicaciones_id_texto=(DSCD-12-PL-4.CODI.)#(P%C3%A1gina12)'


In [ ]:
pages=[
    "https://www.congreso.es/busqueda-de-publicaciones?p_p_id=publicaciones&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&_publicaciones_mode=mostrarTextoIntegro&_publicaciones_legislatura=XIV&_publicaciones_texto=&_publicaciones_id_texto=DSCD-14-PL-1-C1%20DSCD-14-PL-1.CODI.#1",
    "https://www.congreso.es/busqueda-de-publicaciones?p_p_id=publicaciones&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&_publicaciones_mode=mostrarTextoIntegro&_publicaciones_legislatura=XIV&_publicaciones_texto=&_publicaciones_id_texto=DS_P_14_1.CODI.#1"

]

In [2]:
"""
A simple parser to extract all the text of a publication from the body.
It removes any internal script and removes special characters with the str.strip function.
It also gets rid of pagination (Página nnn)
"""
class PublicationParser(HTMLParser):
    #Initializing lists
    lsStartTags = list()
    lsEndTags = list()
    lsStartEndTags = list()
    lsComments = list()
    lsData=list()

    # Indicates when we are inside the body tag
    inBody=False
    # Marker for scripts
    inScript=False

    #HTML Parser Methods
    def handle_starttag(self, startTag, attrs):
        self.lsStartTags.append(startTag)
        if(startTag=="body"):
            self.inBody=True
        if(startTag=="script"):
            self.inScript=True

    def handle_endtag(self, endTag):
        self.lsEndTags.append(endTag)
        if(endTag=="body"):
            self.inBody=False
        elif(endTag=="script"):
            self.inScript=False

    def handle_startendtag(self,startendTag, attrs):
       self.lsStartEndTags.append(startendTag)

    def handle_comment(self,data):
       self.lsComments.append(data)

    def handle_data(self, data):
        if(self.inBody and not self.inScript and data!=''):
            if(not (data.startswith('Página ') or data.startswith('(Página') )):
                self.lsData.append(data.strip())

           


In [3]:
# TODO: a process to scrap all the debates, by now we point to a specific one for testing the code

http = urllib3.PoolManager()
r=http.request("GET",page)
print(r.status)

200


## Main body extractor

Gets the body and finds the start of the debate by looking at the first appearance of the word PRESIDENT*, because when someone speaks its name appears in capital letters and in the case of the chamber president the words PRESIDENTE or PRESIDENTA are used. This appearance usually indicates the start of the interventions.

In [4]:
parser=PublicationParser()
parser.feed(r.data.decode("utf-8"))

def findStart(ls:list)->int:
    result=[]
    index=0
    for line in ls:
        if "PRESIDENT" in line:
            return index
        index+=1
    return -1

index=findStart(parser.lsData)
if(index>=0):
    text=' '.join(parser.lsData[index:])
    print(text[:2500])
else:
    "Error: PRESIDENT not found"

La señora PRESIDENTA: Se reanuda la sesión. Comenzamos con los turnos de intervención de los grupos parlamentarios. Como saben sus señorías, estos intervendrán en orden de mayor a menor, tomando la palabra en último lugar el grupo parlamentario al que pertenece el candidato a la Presidencia del Gobierno. Finalizadas las intervenciones de los grupos parlamentarios, se procederá a la votación, que, como saben, será pública por llamamiento, de acuerdo con los artículos 85 y 86 del Reglamento. Por el Grupo Parlamentario Socialista, tiene la palabra el señor Sánchez Pérez-Castejón. El señor SÁNCHEZ PÉREZ-CASTEJÓN: Gracias, presidenta. Señorías, buenos días. Señor candidato, seré claro y directo. El Grupo Parlamentario Socialista votará en contra de su candidatura a la Presidencia del Gobierno, y lo vamos a hacer con total y absoluto convencimiento, por coherencia con nuestro ideario político, por el compromiso adquirido con nuestros votantes y por el bien de nuestro país (Aplausos), porque 

In [5]:
regexfinder='(?:(?:[A-ZÀ-Ü])(?:-|\s)?)+:'
re.findall(regexfinder,text,re.U|re.M)

['PRESIDENTA:',
 'SÁNCHEZ PÉREZ-CASTEJÓN:',
 'PRESIDENTA:',
 'PRESIDENTA:',
 'SÁNCHEZ PÉREZ-CASTEJÓN:',
 'PRESIDENTA:',
 'PRESIDENTA:',
 'PRESIDENTA:',
 'SÁNCHEZ PÉREZ-CASTEJÓN:',
 'PRESIDENTA:',
 'SÁNCHEZ PÉREZ-CASTEJÓN:',
 'PRESIDENTA:',
 'SÁNCHEZ PÉREZ-CASTEJÓN:',
 'PRESIDENTA:',
 'PRESIDENTA:',
 'IGLESIAS TURRIÓN:',
 'PRESIDENTA:',
 'IGLESIAS TURRIÓN:',
 'PRESIDENTA:',
 'IGLESIAS TURRIÓN:',
 'PRESIDENTA:',
 'DOMÈNECH SAMPERE:',
 'PRESIDENTA:',
 'GARZÓN ESPINOSA:',
 'PRESIDENTA:',
 'FERNÁNDEZ GÓMEZ:',
 'PRESIDENTA:',
 'PRESIDENTA:',
 'IGLESIAS TURRIÓN:',
 'PRESIDENTA:',
 'PRESIDENTA:',
 'IGLESIAS TURRIÓN:',
 'PRESIDENTA:',
 'PRESIDENTA:',
 'PRESIDENTA:',
 'RIVERA DÍAZ:',
 'PRESIDENTA:',
 'RIVERA DÍAZ:',
 'PRESIDENTA:',
 'PRESIDENTA:',
 'RIVERA DÍAZ:',
 'PRESIDENTA:',
 'RIVERA DÍAZ:',
 'PRESIDENTA:',
 'PRESIDENTA:',
 'RIVERA DÍAZ:',
 'PRESIDENTA:',
 'PRESIDENTA:',
 'TARDÀ I COMA:',
 'PSOE:',
 'PRESIDENTA:',
 'PRESIDENTA:',
 'TARDÀ I COMA:',
 'PRESIDENTA:',
 'TARDÀ I COMA:',
 'PRESIDE

In [6]:
# Gets all the indexes of finding an all-caps name (with spaces and special chars)
# followed by a colon symbol. This is the indicator a candidate starts talking
indexes=[(m.start(0),m.end(0)) for m in re.finditer(regexfinder,text, re.U|re.M)]

In [7]:
sentences=pd.DataFrame(columns=['Name','Text'])
last=len(indexes)-1
for i in range(len(indexes)):
    name=text[indexes[i][0]:indexes[i][1]-1]   

    firstIdx=indexes[i][1]+1
    if(i<last):
        lastIdx=indexes[i+1][0]
        while(text[lastIdx]!='.'):
            lastIdx=lastIdx-1
            if(lastIdx==-1):
                break
    else:
        lastIdx=len(text)
    sentence=text[firstIdx:lastIdx]
    sentences.loc[len(sentences)]=[name,sentence]

In [8]:
sentences.to_csv('../debates.csv',index=False)

In [9]:
sentences

,Name,Text
0,PRESIDENTA,Se reanuda la sesión. Comenzamos con los turno...
1,SÁNCHEZ PÉREZ-CASTEJÓN,"Gracias, presidenta. Señorías, buenos días. Se..."
2,PRESIDENTA,Tiene la palabra el candidato a la Presidencia...
3,PRESIDENTA,"Por el Grupo Parlamentario Socialista, tiene l..."
4,SÁNCHEZ PÉREZ-CASTEJÓN,"Gracias, presidenta. Señor Rajoy, no se confun..."
...,...,...
109,PRESIDENTA,Muchas gracias. Tiene la palabra el candidato ...
110,PRESIDENTA,"Muchas gracias. Señorías, en tres minutos come..."
111,PRESIDENTA,"Señorías, ocupen sus escaños. Una vez concluid..."
112,PRESIDENTA,¿Hay alguna señora o señor diputado que no hay...
